In [2]:
!pip install pandas tabulate

In [7]:
import pandas as pd
from tabulate import tabulate

def cargar_datos_excel(archivo_excel, hoja):
    """Cargar datos desde un archivo Excel."""
    return pd.read_excel(archivo_excel, sheet_name=hoja)

def modificar_columnas(df):
    """Modificar el DataFrame para combinar columnas."""
    df.columns = ['TEST CASE ID', 'TEST DATE', 'TEST DESCRIPTION', 'Unnamed: 3', 'TEST CONDITIONS', 'Unnamed: 5', 'SEVERITY']
    return df

def convertir_a_latex(df):
    """Convertir el DataFrame a formato LaTeX."""
    return tabulate(df, headers='keys', tablefmt='latex', showindex=True)

def ajustar_multicolumnas_segunda_fila(tabla_latex):
    """Ajustar las celdas con contenido en dos columnas en la segunda fila de datos."""
    lines = tabla_latex.split('\n')
    hline_count = 0
    for i, line in enumerate(lines):
        if '\\hline' in line:
            hline_count += 1
            if hline_count == 2 and i + 1 < len(lines):
                parts = lines[i + 1].split('&')
                parts[3] = f'\\multicolumn{{2}}{{c}}{{{parts[3].strip()}}}'
                parts[5] = f'\\multicolumn{{2}}{{c}}{{{parts[5].strip()}}}'
                del parts[6]
                del parts[4]
                lines[i + 1] = ' & '.join(parts) + ' \\\\'
                break
    return '\n'.join(lines)

def eliminar_columnas_unnamed(tabla_latex):
    """Eliminar las columnas 'Unnamed: 3' y 'Unnamed: 5' en la fila de encabezado."""
    lines = tabla_latex.split('\n')
    for i, line in enumerate(lines):
        if 'Unnamed: 3' in line and 'Unnamed: 5' in line:
            parts = line.split('&')
            del parts[6]
            del parts[4]
            lines[i] = ' & '.join(parts) + ' \\\\'
            break
    return '\n'.join(lines)

def convertir_a_longtable(tabla_latex):
    """Convertir la tabla a formato longtable e incluir un caption y un label correctamente."""
    # Reemplazar el inicio de la tabla
    tabla_latex = tabla_latex.replace('\\begin{tblr}{colspec={X[1]|X[3]}}', '\\begin{longtable}{|p{2cm}|p{3cm}|p{3cm}|p{3cm}|p{3cm}|}')
    tabla_latex = tabla_latex.replace('\\end{tblr}', '\\end{longtable}')

    # Separar las líneas de la tabla
    lines = tabla_latex.split('\n')
    new_lines = []
    
    step_section = False

    # Procesar las líneas de la tabla
    for i, line in enumerate(lines):
        if 'nan' in line:
            continue
        if '&' in line:
            parts = line.split('&')
            parts = parts[1:]  # Ignorar el primer elemento
            parts = [part.replace('\\multicolumn{2}{c}{', '').replace('}', '').strip() for part in parts]
            if 'TEST CASE ID' in line or 'STEP ID' in line:
                parts = [f'\\textbf{{{part}}}' for part in parts]
                if 'STEP ID' in line:
                    step_section = True
            if step_section and len(parts) > 5:
                parts = parts[:-2]  # Eliminar las últimas dos columnas en la sección de pasos
            new_line = ' & '.join(parts) + ' \\\\ \\hline'
            new_lines.append(new_line)
        else:
            new_lines.append(line)

    # Insertar el caption y el label después del primer \hline
    for i, line in enumerate(new_lines):
        if '\\hline' in line:
            new_lines.insert(i + 1, '\\caption{Descripción de los casos de prueba del módulo Northwest} \\label{tab:casos_prueba} \\\\')
            new_lines.insert(i + 2, '\\hline')
            break

    # Eliminar el \hline justo antes del \caption
    for i, line in enumerate(new_lines):
        if '\\caption' in line and '\\hline' in new_lines[i - 1]:
            del new_lines[i - 1]
            break

    # Insertar encabezados para las páginas siguientes
    for i, line in enumerate(new_lines):
        if '\\endfirsthead' in line:
            new_lines.insert(i + 1, '\\hline')
            new_lines.insert(i + 2, '\\textbf{STEP ID} & \\textbf{STEP DESCRIPTION} & \\textbf{TEST DATE} & \\textbf{EXPECTED RESULTS} & \\textbf{ACTUAL RESULTS} \\\\ \\hline')
            new_lines.insert(i + 3, '\\endhead')
            break

    # Insertar \endfirsthead y \endhead después del encabezado principal
    for i, line in enumerate(new_lines):
        if '\\textbf{TEST CASE ID}' in line and '\\textbf{TEST DATE}' in line:
            new_lines.insert(i + 2, '\\endfirsthead')
            new_lines.insert(i + 3, '\\hline')
            new_lines.insert(i + 4, '\\textbf{STEP ID} & \\textbf{STEP DESCRIPTION} & \\textbf{TEST DATE} & \\textbf{EXPECTED RESULTS} & \\textbf{ACTUAL RESULTS} \\\\ \\hline')
            new_lines.insert(i + 5, '\\endhead')
            break

    return '\n'.join(new_lines)

def limpiar_tabla(tabla_latex):
    """Realizar limpieza y ajustes finales en la tabla LaTeX."""
    tabla_latex = tabla_latex.replace('2024-10-17 00:00:00', '2024-10-17')
    tabla_latex = tabla_latex.replace(r'\textbf{SEVERITY                                                                                                  \\ \\}', r'\textbf{SEVERITY }')
    tabla_latex = tabla_latex.replace(r'ALTA                                                                                                      \\ \\', r'ALTA \\')
    tabla_latex = tabla_latex.replace('\\\\ \\\\', '\\\\')
    tabla_latex = tabla_latex.replace('\\hline\n\\hline', '\\hline')
    return tabla_latex

def guardar_tabla(tabla_latex, archivo_salida):
    """Guardar la tabla LaTeX en un archivo."""
    with open(archivo_salida, 'w') as f:
        f.write(tabla_latex)
    print(f'Tabla LaTeX guardada en "{archivo_salida}"')

# Ejecución del flujo de trabajo
archivo_excel = './tabulate.xlsx'
hoja = 'TC7'
df = cargar_datos_excel(archivo_excel, hoja)
df = modificar_columnas(df)
tabla_latex = convertir_a_latex(df)
tabla_latex = ajustar_multicolumnas_segunda_fila(tabla_latex)
tabla_latex = eliminar_columnas_unnamed(tabla_latex)
tabla_latex = tabla_latex.replace('\\begin{tabular}{rlllllll}', '\\begin{tblr}{colspec={X[1]|X[3]}}')
tabla_latex = tabla_latex.replace('\\end{tabular}', '\\end{tblr}')
tabla_latex_convertida = convertir_a_longtable(tabla_latex)
tabla_latex_convertida = limpiar_tabla(tabla_latex_convertida)
guardar_tabla(tabla_latex_convertida, 'tabla7.tex')

Tabla LaTeX guardada en "tabla7.tex"
